<a href="https://colab.research.google.com/github/LinarKulinar/HPC_SSAU/blob/master/Lab3_Pi_calculation/Lab3_Pi_calculation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Высокопроизводительные вычисления.**

**Лабораторная 3. Вычисление числа π методом Монте-Карло на CPU и GPU.**

Самарский университет

В данном блокноте реализован алгоритм вычисления числа $π$ методом Монте-Карло  для CPU и для GPU.

Для выполнения кода на C на GPU используется библиотека PyCuda.
Для генерации случайных чисел на GPU используется CURAND

Данный код запущен в среде Google Colaboratory

In [139]:
! pip install pycuda

In [140]:
import numpy as np
import time

import pycuda.gpuarray as gpuarray
import pycuda.autoinit
from  pycuda import  driver
from pycuda.compiler import SourceModule

# Алгоритм GPU

- Есть две последовательности: $x, y$ ;
- Считаем $r = x^2 + y^2 $ ;
- Если $r < 1 $, то возвращаем 1 (к счётчику прибавляем 1), иначе возвращаем 0 (оставляем значение счётчика без изменений);
- Выполняется атомарная операция сложения;
- Домножение на $4/n$ будет осуществлено позже.


In [141]:
mod = SourceModule("""
  // переопределил неопределенную функцию atomicAdd(double* address, double val),
  // копипастнув её из документации согласно совету на: 
  // https://forums.developer.nvidia.com/t/why-does-atomicadd-not-work-with-doubles-as-input/56429/7
    #if __CUDA_ARCH__ < 600
  __device__ double atomicAdd(double* address, double val)
  {
      unsigned long long int* address_as_ull =
                                (unsigned long long int*)address;
      unsigned long long int old = *address_as_ull, assumed;

      do {
          assumed = old;
          old = atomicCAS(address_as_ull, assumed,
                          __double_as_longlong(val +
                                __longlong_as_double(assumed)));

      // Note: uses integer comparison to avoid hang in case of NaN (since NaN != NaN)
      } while (assumed != old);

      return __longlong_as_double(old);
  }
  #endif

  __global__ void pi_mc_calc_gpu(double *x, double *y, double *result_gpu, const int n) {
        
        double gpu_count = 0;
        int idx = threadIdx.x + (blockIdx.x*blockDim.x);
        int thread_count = gridDim.x*blockDim.x;

        for (int i=idx; i<n; i += thread_count) {
          int temp;
          temp = pow(x[i], 2) + pow(y[i], 2);
          if (temp < 1)
            gpu_count += 1;
          
        }

        atomicAdd(result_gpu, gpu_count);
  }    
""")

# Алгоритм CPU

- На вход подаются также две последовательности $x,y$ (и число точек $n$);
- Считаем $r = x^2 + y^2 $ ;
- Если $r < 1 $, то возвращаем 1 (к счётчику прибавляем 1), иначе возвращаем 0 (оставляем значение счётчика без изменений);
- Умножение на $4/n$.

In [142]:
def pi_mc_calc_cpu(x, y, n):
  r = x ** 2 + y ** 2
  gen_cpu = [1 for i in range(n) if r[i] < 1]
  result = 4/n * sum(gen_cpu)
  return result

В задании указано, что массив x и y необходимо генерировать на GPU для GPU-вычислений.

Для CPU-вычислений массивы преобразовываются в ndarray функцией get().

In [143]:
 from pycuda.curandom import rand as curand

def generate_data():
  print('Введите число точек: ')
  n = int(input())
  assert n % 16 ==  0, 'вводимое число должно быть кратно 16'
  print('Введенное число точек: ', n)

  x_gpu = curand((n,), dtype=np.double) 
  y_gpu = curand((n,), dtype=np.double)
  x = x_gpu.get().astype(np.double)
  y = y_gpu.get().astype(np.double)
  print('Сгенерированные массивы: \n', x, '\n', y)
  return x, y, n

In [144]:
x, y, n = generate_data()

Введите число точек: 
33554432
Введенное число точек:  33554432
Сгенерированные массивы: 
 [0.74857409 0.77240791 0.51342023 ... 0.21594174 0.22142726 0.69818983] 
 [0.73297373 0.70718343 0.03456352 ... 0.84960281 0.53652539 0.95903933]


In [145]:
cpu_start = time.time()
result_cpu = pi_mc_calc_cpu(x,y, n)
cpu_time = time.time() - cpu_start
print('Число pi: ', result_cpu)
print('Время на CPU: ', round(cpu_time,4))

Число pi:  3.141301989555359
Время на CPU:  15.8091


In [146]:
# т.к. массивы точек являются одномерными
block = (128, 1, 1)
grid = (int(n/(128 * block[0])), 1)

result_gpu = gpuarray.zeros((1,), dtype=np.double)
result_gpu  = result_gpu.get()

In [147]:
calc_gpu = mod.get_function("pi_mc_calc_gpu")

gpu_start = time.time()
calc_gpu(driver.In(x), driver.In(y), driver.Out(result_gpu), np.int32(n), block = block, grid = grid)
driver.Context.synchronize()
gpu_time = time.time() - gpu_start

result_gpu =  result_gpu[0] * 4/n
print('Число pi: ', result_gpu)
print('Время на GPU: ', round(gpu_time,4))

Число pi:  3.141301989555359
Время на GPU:  3.8676


In [148]:
print('Ускорение: ', cpu_time/gpu_time)
print('Сравнение с числом pi: ')
print('GPU:', abs(np.pi -  result_gpu) )
print('CPU:', abs(np.pi -  result_cpu) )

Ускорение:  4.0876067211648195
Сравнение с числом pi: 
GPU: 0.0002906640344342293
CPU: 0.0002906640344342293
